# RAG using RDS SQL Server

Retrieval Augmented Generation is a process that combines retrieval-based models and generative models to enhance natural language generation by retrieving relevant information and incorporating it into the generation process.

In this lab we are going to be writing a simple RAG application code that allows user to ask questions about various wines so they can make a purchasing decision. We will use the semantic search (vector search) capability within RDS SQL Server to retrieve the best matching wine reviews and provide that to LLM for answering user's questions.


### 1. Lab Pre-requisites
a. Download and install python dependencies

b. Download and install Microsoft Open Database Connectivity (ODBC) driver for SQL Server (Linux)

For this notebook we require the use of a few libraries. We'll use the Python clients for SQL Server and SageMaker, and Python frameworks for text embeddings

In [ ]:
# RHEL 7 and Oracle Linux 7
! curl https://packages.microsoft.com/config/rhel/7/prod.repo | sudo tee /etc/yum.repos.d/mssql-release.repo
! sudo yum remove unixODBC-utf16 unixODBC-utf16-devel #to avoid conflicts
! sudo ACCEPT_EULA=Y yum install -y msodbcsql18

# Optional: for bcp and sqlcmd
! sudo ACCEPT_EULA=Y yum install -y mssql-tools18
! echo 'export PATH="$PATH:/opt/mssql-tools18/bin"' >> ~/.bashrc
! source ~/.bashrc

# For unixODBC development headers
! sudo yum install -y unixODBC-devel

# You can ignore pip dependency error that’s displayed towards the end of the installation process 
! pip install --no-build-isolation --force-reinstall \
    "boto3>=1.33.6" \
    "awscli>=1.31.6" \
    "botocore>=1.33.6"

### 2. Import libraries & initialize resource information

The line below will import all the relevant libraries and modules used in this notebook.


In [ ]:
import json
import os
import sys
import boto3
import botocore
import pandas as pd
import pyodbc
import time
import warnings

### 3. Get CloudFormation stack output variables

We have preconfigured a few resources by creating a cloudformation stack in the account. Information of these resources will be used within this lab. We are going to load some of the information variables here.

You can ignore any "PythonDeprecationWarning" warnings.


In [ ]:
# Create a Boto3 session
session = boto3.Session()

# Get the account id
account_id = boto3.client('sts').get_caller_identity().get('Account')

# Get the current region
region = session.region_name

cfn = boto3.client('cloudformation')

# Method to obtain output variables from Cloudformation stack. 
def get_cfn_outputs(stackname):
    outputs = {}
    for output in cfn.describe_stacks(StackName=stackname)['Stacks'][0]['Outputs']:
        outputs[output['OutputKey']] = output['OutputValue']
    return outputs

# Setup variables to use for the rest of the demo
cloudformation_stack_name = "genai-data-foundation-workshop"

outputs = get_cfn_outputs(cloudformation_stack_name)
rds_host = outputs['RDSSQLServerEndpoint']
s3_bucket = outputs['s3BucketTraining']
bedrock_inf_iam_role = outputs['BedrockBatchInferenceRole']
bedrock_inf_iam_role_arn = outputs['BedrockBatchInferenceRoleArn']
sagemaker_notebook_url = outputs['SageMakerNotebookURL']

# We will just print all the variables so you can easily copy if needed.
outputs

### 4. Prepare data
Below is the code that loads dataset of wine reviews, we'll use this data set to recommend wines that resemble the user provided description.

#### Sampling subset of the records to load into opensearch quickly
Since the data is composed of 129,000 records, it could take some time to convert them into vectors and load them in a vector store. Therefore, we will take a subset (300 records) of our data. We will add a variable called record_id which corresponds to the index of the record

In [ ]:
url = "https://raw.githubusercontent.com/davestroud/Wine/master/winemag-data-130k-v2.json"
df = pd.read_json(url)
columns = ['points', 'description', 'designation', 'variety', 'country', 'winery', 'title']
df_sample = df[columns]
df_sample = df_sample.sample(300,random_state=37).reset_index()
df_sample['record_id'] = range(1, len(df_sample) + 1)
df_sample[:5]

Removing single quotes from the columns 

In [ ]:
# Remove all single quotes from the following columns:
df_sample['description'] = df_sample['description'].astype(str).str.replace("[']", "", regex=True)
df_sample['designation'] = df_sample['designation'].astype(str).str.replace("[']", "", regex=True)
df_sample['variety'] = df_sample['variety'].astype(str).str.replace("[']", "", regex=True)
df_sample['country'] = df_sample['country'].astype(str).str.replace("[']", "", regex=True)
df_sample['winery'] = df_sample['winery'].astype(str).str.replace("[']", "", regex=True)
df_sample['title'] = df_sample['title'].astype(str).str.replace("[']", "", regex=True)

## 5. Using Amazon Bedrock Titan embedding to convert text to vectors
Amazon Bedrock offers Amazon Titan embedding v2 model that generates vector embeddings for text. This model will be used as our primary model for embeddings.

#### Helper method to invoke Titan embedding model in Amazon Bedrock
Creating a helper method in python to invoke Amazon Titan embedding model from Amazon Bedrock to generate embeddings. We will update `df_sample` data frame and add a new column called `embedding` in it. Once this cell is executed, our data frame will be ready to load into opensearch.

In [ ]:
import boto3
import pandas as pd
import os
from typing import Optional

# External Dependencies:
import boto3
from botocore.config import Config


bedrock_client = boto3.client(
    "bedrock-runtime", 
    region, 
    endpoint_url=f"https://bedrock-runtime.{region}.amazonaws.com"
)


def add_embeddings_to_df(df, text_column):

    # Create an empty list to store embeddings
    embeddings = []

    # Iterate over the text in the specified column
    for text in df[text_column]:
        embedding = embed_phrase(text)
        embeddings.append(embedding)
        

    # Add the embeddings as a new column to the DataFrame
    df['embedding'] = embeddings

    return df

def embed_phrase( text ):
        
    model_id = "amazon.titan-embed-text-v2:0"  # 
    accept = "application/json"
    contentType = "application/json"

    # Prepare the request payload
    request_payload = json.dumps({"inputText": text})


    response = bedrock_client.invoke_model(body=request_payload, modelId=model_id, accept=accept, contentType=contentType)

    # Extract the embedding from the response
    response_body = json.loads(response.get('body').read())


    # Append the embedding to the list
    embedding = response_body.get("embedding")
    return embedding

df_sample = add_embeddings_to_df(df_sample, 'description')

df_sample[:5]

#### Let's try to create an embedding of a simple input text
You can see its an array of floating point numbers. While it does not make sense to human eye/brain, this array of numbers captures the semantics and knowledge of the text and that can be later used to compare two different text blocks. This method will be used to convert our query to a vector representation.

In [ ]:
## Create an vector embedding for input text
input_text = "A wine that pairs well with turkey breast?"

embedding = embed_phrase(input_text)

#printing text and embedding

print(f"{input_text=}")

#only printing first 10 dimensions of the 1024 dimension vector 
print(f"{embedding[:10]=}")

### 6. Create a connection to RDS SQL Server

Next, we'll use Python API to set up connection with RDS SQL Server instance, our vector database for this workshop.

#### Retrieving credentials from Secrets manager

To avoid hard coding the user name and password in our code, we have dynamically generated a username and password at the time of deploying the cluster. This user name and password is stored in AWS Secrets Manager service. We will retrieve secret from secrets manager to establish database connection.

In [ ]:
kms = boto3.client('secretsmanager')
db_credentials = json.loads(kms.get_secret_value(SecretId=outputs['DBSecret'])['SecretString'])

# For this lab we will use credentials that we have already created in AWS Secrets manager service. Secrets
# manager service allows you to store secrets securily and retrieve it through code in a safe manner.

dbuser = db_credentials['username']
dbpass = db_credentials['password']
print(dbuser,dbpass)


#### Enable and Verify SageMaker Notebook instance to RDS connectivity

Sagemaker instance and RDS SQL Server instance are deployed in different VPCs for the workshop. Enable the ingress rules to allow remote traffic and test the connectivity using sample python program below as part of preparation work.


In [ ]:
# Execute this to add the ingress rules to allow remote access to PostgreSQL
!aws ec2 authorize-security-group-ingress --group-name default --protocol tcp --port 1433 --cidr 0.0.0.0/0

In [ ]:
from warnings import filterwarnings
import pyodbc
filterwarnings("ignore", category=UserWarning, message='.*pandas only supports SQLAlchemy connectable.*')

try:
    cnxn = pyodbc.connect( f'DRIVER=ODBC Driver 18 for SQL Server;SERVER={rds_host};DATABASE=master;UID={dbuser};PWD={dbpass};Encrypt=no')
    print (cnxn)
    print ("Test connection works.")
    cnxn.close()
except Exception as e:
    print("I am unable to connect to the database!\n")

    print (e)


### 7. Preparing the raw data for insertion and the tables in RDS SQL Server

In this step, we will prepare the raw data to be inserted into RDS. We will create the VectorDB and tables to store them. Final step would be to insert the data into the table

In [ ]:
# Map the Pandas Dataframe columns to the database table column names
column_map = {
    'index':'id',
    'points': 'points',
    'description': 'description',
    'designation': 'designation',
    'variety': 'variety',
    'country': 'country',
    'winery': 'winery',
    'title': 'wine_name',
    'record_id':'record_id',
    'embedding': 'content_vector',
}

# Rename pandas dataframe columns
df_sample.rename(columns=column_map, inplace=True)
df_sample[:5]

In [ ]:
# Create the VectorDB on RDS SQL Server
cnxn = pyodbc.connect( f'DRIVER=ODBC Driver 18 for SQL Server;SERVER={rds_host};DATABASE=master;UID={dbuser};PWD={dbpass};Encrypt=no')
cursor = cnxn.cursor()
sql_stm = '''
drop database if exists vectordb;
create database vectordb;
'''
cnxn.autocommit = True
cursor.execute(sql_stm)
cnxn.commit()

In [ ]:
# Create the target database table
cnxn = pyodbc.connect( f'DRIVER=ODBC Driver 18 for SQL Server;SERVER={rds_host};DATABASE=vectordb;UID={dbuser};PWD={dbpass};Encrypt=no')
cursor = cnxn.cursor()
sql_stm = '''
drop table if exists dbo.wines_embedding_bedrock;
CREATE TABLE [dbo].[wines_embedding_bedrock](
    [id] [int],
    [points] [int] NOT NULL,
    [description] [varchar](max) NOT NULL,
    [designation] [varchar](1000) NOT NULL,
    [variety] [varchar](100) NOT NULL,
    [country] [varchar](100) NOT NULL,
    [winery] [varchar](100) NOT NULL,
    [wine_name] [varchar](500) NOT NULL,
    [record_id][int] NOT NULL,
    [content_vector] [varchar](max) NOT NULL
) ON [PRIMARY]
'''
cursor.execute(sql_stm)
cnxn.commit()

In [ ]:
# Prepare data for insertion
data = df_sample.to_dict(orient='records')

# Insert data into the database table.
cnxn = pyodbc.connect( f'DRIVER=ODBC Driver 18 for SQL Server;SERVER={rds_host};DATABASE=vectordb;UID={dbuser};PWD={dbpass};Encrypt=no')

cursor = cnxn.cursor()
sql_stm = '''TRUNCATE TABLE dbo.wines_embedding_bedrock'''
cursor.execute(sql_stm)
cnxn.commit()

with cnxn.cursor() as cursor:
    for row in data:
        values = ', '.join(f"'{value}'" for value in row.values())
        columns = ', '.join(row.keys())
        query = f"INSERT INTO wines_embedding_bedrock ({columns}) VALUES ({values})"
        # print(query)
        cursor.execute(query)
    cnxn.commit()

To verify that we have inserted 300 records into the table

In [ ]:
cnxn = pyodbc.connect( f'DRIVER=ODBC Driver 18 for SQL Server;SERVER={rds_host};DATABASE=vectordb;UID={dbuser};PWD={dbpass};Encrypt=no')
sql_query = pd.read_sql("select count (1) as total_rows from wines_embedding_bedrock", cnxn)
df = pd.DataFrame(sql_query, columns=['total_rows'])
display(df)

### 8. Create Vector table and Columnstore Index

Next, we will create a vector table and a Columnstore Index. SQL Server Columnstore Index offer built-in optimizations including SIMD and AVX-512 that accelerate vector operations. 

In [ ]:
# Create a Vector table and a columnstore index
cnxn = pyodbc.connect( f'DRIVER=ODBC Driver 18 for SQL Server;SERVER={rds_host};DATABASE=vectordb;UID={dbuser};PWD={dbpass};Encrypt=no')
cursor = cnxn.cursor()
sql_stm = '''
drop table if exists dbo.wines_content_vector;
with cte as
(
    select
        v.id as index_id,
        cast(tv.[key] as int) as vector_value_id,
        cast(tv.[value] as float) as vector_value
    from
        [dbo].[wines_embedding_bedrock] as v
    cross apply
        openjson(content_vector) tv
)
select
    index_id,
    vector_value_id,
    vector_value
into
    dbo.wines_content_vector
from
    cte;

create clustered columnstore index ixc
on dbo.wines_content_vector;
'''
#print (sql_stm)
cursor.execute(sql_stm)
cnxn.commit()

### 9. Create Similarity Search function

Next, we create a user defined function (UDF) to support similarity search based on cosine distance


In [ ]:
# Create UDF for Similarity search
cnxn = pyodbc.connect( f'DRIVER=ODBC Driver 18 for SQL Server;SERVER={rds_host};DATABASE=vectordb;UID={dbuser};PWD={dbpass};Encrypt=no')
cursor = cnxn.cursor()
sql_stm = '''drop function if exists [dbo].[SimilarWines]'''
sql_stm2 = '''
create   function [dbo].[SimilarWines](@vector nvarchar(max))
returns table
as
return with cteVector as
(
    select 
        cast([key] as int) as [vector_value_id],
        cast([value] as float) as [vector_value]
    from 
        openjson(@vector)
),
cteSimilar as
(
select top (50)
    v2.index_id, 
    sum(v1.[vector_value] * v2.[vector_value]) as cosine_distance -- Optimized as per https://platform.openai.com/docs/guides/embeddings/which-distance-function-should-i-use
from 
    cteVector v1
inner join 
    dbo.wines_content_vector v2 on v1.vector_value_id = v2.vector_value_id
group by
    v2.index_id
order by
    cosine_distance desc
)
select 
    a.points,
    a.description,
    a.designation,
    a.variety,
    a.country,
    a.winery,
    a.wine_name,
    r.cosine_distance
from 
    cteSimilar r
inner join 
    dbo.wines_embedding_bedrock a on r.index_id = a.id
'''
cursor.execute(sql_stm)
cursor.execute(sql_stm2)
cnxn.commit()


### 10. Search vector with "Semantic Search"

Now we can define a helper function to execute the search query for us to find a wine whose review most closely matches the requested description. retrieve_sql_with_semantic_search embeds the search phrase, searches the table in RDS SQL Server for the closest matching vector, and returns the top result.


In [ ]:
def retrieve_sql_with_semantic_search(phrase):
    n=3
    search_vector = embed_phrase(phrase)
    json_query_result = json.dumps(search_vector)

    stmt = '''select top (?) points, description, designation, variety, country, winery, wine_name, cosine_distance \
    from dbo.SimilarWines(?) as r order by cosine_distance desc'''
    results = []
    
    with pyodbc.connect( f'DRIVER=ODBC Driver 18 for SQL Server;SERVER={rds_host};DATABASE=vectordb;UID={dbuser};PWD={dbpass};Encrypt=no') as cnxn:
        curs = cnxn.cursor()
        curs.execute (stmt,(n,json_query_result))
    
        for record in curs:
            result = {
                "points":record[0],
                "description":record[1],
                "designation":record[2],
                "variety":record[3],
                "country":record[4],
                "winery":record[5],
                "wine_name":record[6]
                }
            results.append(result)
    
    return results

Use the semantic search to get similar records with the sample question

In [ ]:
question_on_wine="Best Australian wine that goes great with steak?"

example_request = retrieve_sql_with_semantic_search(question_on_wine)
print(json.dumps(example_request, indent=4))

### 11.  Prepare a method to call Amazon Bedrock - Anthropic Claude Sonnet model

Now we will define a function to call LLM to answer user's question. As LLM is trained with static data, and it does not have our wine review knowledge. While it may be able to answer, it may not be an answer that a business prefers. For example. in our case, we would not want to recommend a wine that we do not stock. So the recommendation has to be one of the wines from our collection i.e. 300 reviews that we loaded.

After defining this function we will call it to see how LLM answers questions without the wine review data.


In [ ]:
def query_llm_endpoint_with_json_payload(encoded_json):

    # Create a Bedrock Runtime client
    bedrock_client = boto3.client('bedrock-runtime')
    # Set the model ID for Claude 3 Sonnet
    model_id = 'anthropic.claude-3-sonnet-20240229-v1:0'
    accept = 'application/json'
    content_type = 'application/json'


    try:
        # Invoke the model with the native request payload
        response = bedrock_client.invoke_model(
            modelId=model_id,
            body=str.encode(str(encoded_json)),
            accept = accept,
            contentType=content_type
        )

        # Decode the response body
        response_body = json.loads(response.get('body').read())
        return response_body
    except Exception as e:
        print(f"Error: {e}")
        return none

def query_llm(system, user_question):
    # Define the prompt for the model
    prompt = "Write a sonnet about the beauty of nature."

    # Prepare the model's payload
    payload = json.dumps({
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 10000,
        "system": system,
        "messages": [
            {
              "role": "user",
              "content": [
                {
                  "type": "text",
                  "text": f"{user_question}"
                }
              ]
            }
          ]
        })
    


    query_response = query_llm_endpoint_with_json_payload(payload)

    return query_response['content'][0]['text']


Let's check the generated result for a wine recommendation. It may not be one of the wine that we stock.

In [ ]:
def query_llm_without_rag(question):
    
    # Claude model has 2 parts of the prompt. System prompt guides the model what role to play
    system_prompt = f"You are a sommelier that uses their vast knowledge of wine to make great recommendations people will enjoy."
    
    # User prompt is the engineer prompt that has the context that model should reference to answer questions
    user_prompt = (
        f" As a sommelier, you must include the wine variety, the country of origin, "
        f"and a colorful description relating to the customer question."
        f"\n Customer question: {question}"
        f"\n Please provide name of the wine at the end of the answer, in a new line, in format Wine name: <wine name>"
    )
    return query_llm(system_prompt, user_prompt)


question_on_wine="Best Australian wine that goes great with steak?"

print(f"The recommened wine from LLM without RAG: \n{query_llm_without_rag(question_on_wine)}\n")

#### Testing for hallucination.

Let's copy the wine name from the last line and past it in the question variable below to see if we have this wine in our stock. Please review the list of wines that are returned. They may be from portugal but not exactly the one we have been recommended by the model.

Note: If you do not see the same wine name in the wine_name variable below, you should replace it so we can verify that the wine recommended is not in our RDS SQL Server table.


In [ ]:
wine_name = "Penfolds Grange Shiraz"
example_request = retrieve_sql_with_semantic_search(wine_name)
print("Matching wine records in our reviews:")
print(json.dumps(example_request, indent=4))

### 12. Retrieval Augmented Generation

To resolve LLM hallunination problem, we can more context to LLM so that LLM can use context information to fine the model and generated factual result. RAG is one of the solution to the LLM hallucination.
Create a prompt for the LLM using the search results from RDS SQL Server

We will be using the Anthropic Sonnet model with one-shot prompting technique. Within instructions to the model in the prompt, we will provide a sample wine review and how model should use to answer user's question. At the end of the prompt wine reviews retrieved from RDS SQL Server will be included for model to use.

Before querying the model, the below function generate_sql_based_system_prompt is used to put together user prompt. The function takes in an input string to search the wines_content_vector table for a matching wine, then compose the user prompt for LLM.

System prompt defines the role that LLM plays.

User prompt contains the instructions and the context information that LLM model uses to answer user's question.

The prompt is in the following format:

#### SYSTEM PROMPT:

    You are a sommelier that uses their vast knowledge of wine to make great recommendations people will enjoy. 

#### USER PROMPT

    As a sommelier, you must include the wine variety, the country of origin, and a colorful description relating to the user's question.

    Data:{'description': 'This perfumey white dances in intense and creamy layers of stone fruit and vanilla, remaining vibrant and balanced from start to finish. The generous fruit is grown in the relatively cooler Oak Knoll section of the Napa Valley. This should develop further over time and in the glass.', 'winery': 'Darioush', 'points': 92, 'designation': None, 'country': 'US'}

    Recommendation:I have a wonderful wine for you. It's a dry, medium bodied white wine from Darioush winery in the Oak Knoll section of Napa Valley, US. It has flavors of vanilla and oak. It scored 92 points in wine spectator.

    Data: {retrieved_documents}

    Question from the user as is

#### Package the prompt and query the LLM

We will create a final function to query the LLM with the prompt. query_llm_with_rag is a function that calls LLM in a RAG.

query_llm_with_rag combines everything we've done in this module. It does all of the following:

- searches the RDS SQL Server vector data with semantic search for the relevant wine with "description vector"
- generate an LLM prompt from the search results
- queriy the LLM with RAG for a response



In [ ]:
def query_llm_with_rag(user_question):
    retrieved_documents = retrieve_sql_with_semantic_search(user_question)
    one_shot_description_example = "{'description': 'This perfumey white dances in intense and creamy layers of stone fruit and vanilla, remaining vibrant and balanced from start to finish. The generous fruit is grown in the relatively cooler Oak Knoll section of the Napa Valley. This should develop further over time and in the glass.', 'winery': 'Darioush', 'points': 92, 'designation': None, 'country': 'US'}"
    one_shot_response_example = "I have a wonderful wine for you. It's a dry, medium bodied white wine from Darioush winery in the Oak Knoll section of Napa Valley, US. It has flavors of vanilla and oak. It scored 92 points in wine spectator."
    system_prompt= "You are a sommelier that uses vast knowledge of wine to make great recommendations people will enjoy"
    user_prompt = (
        f"As a sommelier, you must include the wine variety, the country of origin, and a colorful description relating to the user question. You are must pick a wine in \"Wine data\" section only, one that matches best the customer question. Do not suggest anything outside of the wine data provided. You don't necessarily have to pick the top rated wine if its not best suited for customer question.\n"
        f"Wine data: {one_shot_description_example} \n Recommendation: {one_shot_response_example} \n"
        f"Wine data: {retrieved_documents} \n"
        f"Customer Question: {user_question} \n"        
    )
    response = query_llm(system_prompt, user_prompt)
    return response

And finally, let's call the function and get a wine recommendation.

In [ ]:
question_on_wine="Best Australian wine that goes great with steak?"
recommendation = query_llm_with_rag(question_on_wine)
print(recommendation)

print(f"\n\ndocuments retrieved for above recommendations were \n\n{json.dumps(retrieve_sql_with_semantic_search(question_on_wine), indent=4)}")

Let's change it to Italian wine - it should produce a matching result.

We will call the same method again to see if there is an italian wine in our catalog.


In [ ]:
question_on_wine="Best Italian wine that goes great with steak?"
recommendation = query_llm_with_rag(question_on_wine)
print(recommendation)

print(f"\n\ndocuments retrieved for above recommendations were \n\n{json.dumps(retrieve_sql_with_semantic_search(question_on_wine), indent=4)}")

You might notice that we asked for Australian wines that goes well with steak and we do not have any such wine in our collection. Therefore the model politely excuses. You may change the question and see how LLM recommends a wine from our select list that best suites your question.

### Conclusion
In this lab you built a simple wine recommendation chatbot. In this particular lab you used Amazon Bedrock titan v2 model to create vector embedding for our data. Then we loaded this data in OpenSearch index with `description_vector` field. At search time, we used Amazon Titan v2 model again to convert our query question into vector embedding and used semantic search to retrieve results. These results were then passed on to Anthropic Claude Sonnet 3 model which was able to recommend us a wine from within our catalog.

## Lab finished - you may now go back to lab instructions section